In [ ]:
# default_exp mgmnt.prep.i

# Preprocessing

> This module comprises all preprocessing techniques applied to software artifacts:
>
>> Text-based Artifacts: Classical preprocessing (stemming, lemas, etc) and BPE Binary Artifacts:
>
>> To Do Vision-based Artifacts:
>
>> To Do Parsing: Techniques to control and manipulate source code (complete with deep generator project)

In [ ]:
! pip install dit

In [ ]:
# export
# Imports
import pandas as pd
import sentencepiece as sp

from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
def jsonl_list_to_dataframe(file_list, columns=None):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f,
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

In [ ]:
# export
def get_dfs(path):
    """
        Grabs the different data splits and converts them into dataframes.
        Expects format from Code Search Net Challenge.
    """
    dfs = []
    for split in ["train", "valid", "test"]:
        files = sorted((path/split).glob("**/*.gz"))
        df = jsonl_list_to_dataframe(files, ["code", "docstring"])
        dfs.append(df)
        
    return dfs

In [ ]:
path = Path('/tf/data/')

In [ ]:
df_trn, df_val, df_tst = get_dfs(path/"java/final/jsonl")
df_trn.head()

code  \
0  protected final void bindIndexed(Configuration...   
1  public void setServletRegistrationBeans(\n\t\t...   
2  public void addServletRegistrationBeans(\n\t\t...   
3  public void setServletNames(Collection<String>...   
4  public void addServletNames(String... servletN...   

                                           docstring  
0  Bind indexed elements to the supplied collecti...  
1  Set {@link ServletRegistrationBean}s that the ...  
2  Add {@link ServletRegistrationBean}s for the f...  
3  Set servlet names that the filter will be regi...  
4  Add servlet names for the filter.\n@param serv...

In [ ]:
# Save some test data
df_trn.sample(frac = 0.01).to_csv('./test_data/trn.csv', index = False)
df_val.sample(frac = 0.01).to_csv('./test_data/val.csv', index = False)
df_tst.sample(frac = 0.01).to_csv('./test_data/tst.csv', index = False)

In [ ]:
# export
def df_to_txt_file(df, output, cols):
    """Converts a dataframe and converts it into a text file that SentencePiece can use to train a BPE model"""
    if cols is None: cols = list(df.columns)
    merged_df = pd.concat([df[col] for col in cols])
    
    with open(output/'text.txt', 'w') as f:
        f.write('\n'.join(list(merged_df)))
    return output/'text.txt'

In [ ]:
# export
def sp_model_from_df(df, output, model_name, cols = None):
    """Trains a SentencePiece BPE model from a pandas dataframe"""
    fname = df_to_txt_file(df, output, cols)
    sp.SentencePieceTrainer.train(f'--input={fname} --model_prefix={output / model_name} --hard_vocab_limit=false')

In [ ]:
# export
def sp_model_from_glob(path, glob, model_name):
    fns = list(path.glob(glob))
    fns = ",".join(map(str, fns))
    sp.SentencePieceTrainer.train(f'--input={fns} --model_prefix={path / model_name} --hard_vocab_limit=false')

In [ ]:
# export
def gen_hugface_model(df, output, tokenizer = ByteLevelBPETokenizer(), vocab_sz = 30_000, min_freq = 3, cols = None):
    fname = df_to_txt_file(df, output, cols)
    tokenizer.train(files = [str(fname)], vocab_size = vocab_sz, min_frequency = min_freq, special_tokens=[
        "<s>",
        "<pad>",
        "</s>",
        "<unk>",
        "<mask>",
    ])
    
    return tokenizer

In [ ]:
path = Path("./test_data")
model_name = "test"

In [ ]:
df = pd.read_csv(path / 'trn.csv')
df.head()

code  \
0  private static void createCode(String packageN...   
1  @Override\n    public void flushCache() {\n   ...   
2  public void addRule(IntDependency dependency, ...   
3  @Override\n  public boolean removeIfEquals(K k...   
4  public void marshall(DatasetContentDeliveryDes...   

                                           docstring  
0                                    Create the Java  
1                                           LI3492-2  
2  Add this dependency with the given count to th...  
3                  Remove the object from the cache.  
4               Marshall the given parameter object.

In [ ]:
tokenizer = gen_hugface_model(df, path)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

In [ ]:
print(tokenizer.encode("public static void main(String[] args) { getDirFromLib(); }").tokens)

['<s>', 'public', 'Ġstatic', 'Ġvoid', 'Ġmain', '(', 'String', '[]', 'Ġargs', ')', 'Ġ{', 'Ġget', 'Dir', 'From', 'Lib', '();', 'Ġ}', '</s>']


In [ ]:
tokenizer.save(str(path), "java_tokenizer")

['test_data/java_tokenizer-vocab.json', 'test_data/java_tokenizer-merges.txt']

In [ ]:
dummy_data = {
        'first': ['1', '2', '6', '7', '8'],
        'second': ['K', 'M', 'O', 'Q', 'S'],
        'third': ['L', 'N', 'P', 'R', 'T']}

In [ ]:
df = pd.DataFrame(dummy_data2); df

id Feature1 Feature2
0  1        K        L
1  2        M        N
2  6        O        P
3  7        Q        R
4  8        S        T

In [ ]:
df_to_txt_file(df, Path('./test_data'), list(df.columns))

PosixPath('test_data/text.txt')

In [ ]:
path = Path("./test_data")
model_name = "test"

In [ ]:
sp_model_from_dfs(df, path, model_name, list(df.columns))

In [ ]:
spm = sp.SentencePieceProcessor()
spm.Load(str(path/f"{model_name}.model"))

True

In [ ]:
spm.EncodeAsPieces("Hello, world!")

['▁', 'Hello,', '▁', 'world!']

In [ ]:
# export
def tokenize_fns(fns, tokenizer, exts, output, data_type):
    docs = []
    for fn in fns:
        system = fn.parent.name
        output_path = output/system/data_type
        output_path.mkdir(parents=True, exist_ok=True)
        files = []
        for ext in exts:
            files.extend(fn.glob(f'**/*.{ext}'))
        for file in files:
            if 'README' not in file.name:
                with open(file, encoding='ISO-8859-1') as f:
                    docs.append(tokenizer.EncodeAsPieces(f.read()))
                with open((output_path/file.name).with_suffix('.bpe'), 'w') as f:
                    f.write(' '.join(docs[-1]))
            
    return docs

In [ ]:
# export
def read_bpe_files(path):
    bpe_files = []
    for file in path.glob('**/*.bpe'):
        with open(file) as f:
            bpe_files.append(f.read().split(' '))
    
    return bpe_files

In [ ]:
# export
def split_lines_to_files(lines, fn_pattern, output_path, tokenizer):
    for line in lines:
        fn, content = line.split(fn_pattern)
        fn = fn.replace('"', '')
        fn = fn.replace(' Test ', '')
        content = tokenizer.EncodeAsPieces(content)
        with open((output_path/fn).with_suffix('.bpe'), 'w') as f:
                    f.write(' '.join(content))

In [ ]:
path = Path('../benchmarking/traceability/')

In [ ]:
spm = sp.SentencePieceProcessor()
spm.Load(str(path/'datasets/italian/italian_bpe.model'))

True

In [ ]:
ebt_path = path/'datasets/italian/ebt'; ebt_path

PosixPath('../benchmarking/traceability/datasets/italian/ebt')

In [ ]:
with open(ebt_path/'[ebt-raw-req].txt') as f:
    split_lines_to_files(f.read().split('\n')[:-1], '\t', path/'testbeds/bpe/italian/ebt/req', spm)

In [ ]:
with open(ebt_path/'[ebt-raw-tc].txt') as f:
    split_lines_to_files(f.read().split('\n')[:-1], 'case:', path/'testbeds/bpe/italian/ebt/tc', spm)

In [ ]:
!ls

00_data.preprocessing.ipynb	 07_model.train.ipynb
01_data.exploratory.ipynb	 08_data.exploratory.information.ipynb
02_data.management.ipynb	 09_data.exploratory.stats.ipynb
03_data.representation.ipynb	 10_data.exploratory.visualize.ipynb
04_templates.example.ipynb	 index.ipynb
05_blog.example.ipynb		 test_data
06_benchmark.traceability.ipynb


In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_data.preprocessing.ipynb.
Converted 01_data.exploratory.ipynb.
Converted 02_data.management.ipynb.
Converted 03_data.representation.ipynb.
Converted 04_templates.example.ipynb.
Converted 05_blog.example.ipynb.
Converted 06_benchmark.traceability.ipynb.
Converted 07_model.train.ipynb.
Converted 08_data.exploratory.information.ipynb.
Converted 09_data.exploratory.stats.ipynb.
Converted 10_data.exploratory.visualize.ipynb.
Converted index.ipynb.
